Step 1: Scrape Product Links

In [ ]:
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from seleniumwire import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

import requests
from bs4 import BeautifulSoup

In [ ]:
#Step 0: Set up proxy server
username = "brd-customer-hl_ab73b620-zone-datacenter10"
password = "jz8qvx63tljn"
proxy_host = "brd.superproxy.io"
proxy_port = "22225"

proxy_options = { 
    'proxy': { 
        'http': f'http://{username}:{password}@{proxy_host}:{proxy_port}', 
        'https': f'https://{username}:{password}@{proxy_host}:{proxy_port}', 
        'no_proxy': 'localhost, 127.0.0.1'  # Bypass proxy for local addresses 
    } 
} 

# Step 1: Open website and close any window
driver2 = webdriver.Chrome(seleniumwire_options=proxy_options)
driver2.get('https://oliveyoung.com//')
print(driver2.title)
time.sleep(1)

 # Step 2: Close the 'Are you resident of CA?' window
try:
    resident_YN = WebDriverWait(driver2, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[15]/div[2]/div/div[2]/div/button")) )
    time.sleep(2)
    resident_YN.click()
    print("Cleared 'Are you a CA resident' question")
except:
    print("No resident CA question shown.")


# Step 3: Click the Catergories button -> choose Skincare
categories_button = driver2.find_element(By.LINK_TEXT,"Categories")
categories_button.click()
time.sleep(2)

skincare_button2 = driver2.find_element(By.LINK_TEXT,"Face Masks")
skincare_button2.click()
time.sleep(2)

# Step 4: Get total page numbers
more_page_button = driver2.find_element(By.CSS_SELECTOR,".btn-page-more")
driver2.execute_script("arguments[0].scrollIntoView(true)",more_page_button)
pages_text = more_page_button.text
page_start_index = pages_text.index("/")+1
page_end_index = len(pages_text)-1
total_pages = int(pages_text[page_start_index:page_end_index])
time.sleep(2)

# Step 5: Scroll down to the 'More page' button and click it 'Total_pages' times
for page in range(total_pages):
    try:
        more_page_button = driver2.find_element(By.CSS_SELECTOR,".btn-page-more")
        more_page_button.click()
        time.sleep(4)
        WebDriverWait(driver2, 5).until(EC.presence_of_element_located((By.CLASS_NAME,"unit-list")) )
        
    except Exception as e:
        print(f"Exception: {e}")
        print("Loading page did not complete.")
        break
     
print(f"Loading page {total_pages} completed.")

# Step 6: Extract href links of products 
product_links = []
product_count = 0
all_links = driver2.find_elements(By.TAG_NAME,"a")

for link in all_links:
    all_hrefs = link.get_attribute("href")
    if all_hrefs and "/product/" in all_hrefs:
        product_count += 1
        product_links.append(all_hrefs)

    # Step 6.1: There are 2 href links for each product, so I have to remove duplicates
def distinct(product_links):
    distinct_links = []
    for l in product_links:
        if l not in distinct_links:
            distinct_links.append(l)
    return distinct_links

print(f"All Product links found: ")   
distinct_list = distinct(product_links)
print(distinct_list)
print(f"Total link count: {len(distinct_list)}") 


   # Step 6.2: Export product links to a .csv file
df = pd.DataFrame(distinct_list, columns=['Link'])
df.to_csv('ProductLinks-masks.csv')


driver2.quit()

In [ ]:
# Step 2: Go to link and collect product data
def get_product_data (driver3, prod_link):   
    products = []
    users = []
    count = 0
    for link in prod_link:
        retries = 0
        success = False
        
        while retries <= 3 and not success:
            try:
                prod_data = {'status': None,
                        'brand': None,
                        'prod_name': None,
                        'price': None,
                        'rating': None,
                        'category':None,
                        'subcategory': None,
                        'ingredients': None,
                        'reviews_no': None,
                        'review': None,
                        'link': None
                        }

                user_data = {'prod_name': None,
                        'user_skin_type': None,
                        'user_skin_concerns' : None
                        }
                
                #driver3.set_window_size(1920,1080)    # #If compile in 1920 x 1080 res screen
                driver3.get(link)
                time.sleep(2)
                prod_data['brand'] = get_brand_name(driver3)
                prod_data['prod_name'] = get_product_name(driver3)
                user_data['prod_name'] = get_product_name(driver3)
                prod_data['price'] = get_price(driver3)
                prod_data['rating'] = get_rating(driver3) 
                prod_data['reviews_no'] = get_review_no(driver3)
                prod_data['category'], prod_data['subcategory'] = get_category(driver3)
                prod_data['ingredients'] = get_ingredients(driver3)
                prod_data['link'] = link

                review_pages = get_review_pages(driver3)
                if review_pages > 0:
                    #translate_review(driver3)
                    load_review_pages(driver3, review_pages)
                    expand_review(driver3)
                    prod_data['review'] = extract_review(driver3)
                    user_data['user_skin_type'] = extract_user_skin_type(driver3)
                    user_data['user_skin_concerns'] = extract_user_skin_concerns(driver3)

                elif review_pages == 0 and prod_data['reviews_no'] > 0:
                    #translate_review(driver3)
                    expand_review(driver3)
                    prod_data['review'] = extract_review(driver3)
                    user_data['user_skin_type'] = extract_user_skin_type(driver3)
                    user_data['user_skin_concerns'] = extract_user_skin_concerns(driver3)
                    
                else:
                    prod_data['review'] = 0
                    user_data['user_skin_type'] = 0
                    user_data['user_skin_concerns'] = 0
                    print("Zero review. No loading and translation required.")
                products.append(prod_data)
                users.append(user_data)  
                count += 1
                success = True
                print(f"COMPLETED DATA EXTRACTION FROM {count} PRODUCT(S).")

            except TimeoutException:
                retries += 1
                if retries == 3:
                    print(f"Could not load link {count}th.")
                    pass
                continue

            except NoSuchElementException:
                success = False
                print("Page not found or no longer exists.")
                break
    return products, users
    

    # Step 2.1: Extract brand and product name
def get_brand_name(driver): 
    try:
        brand_info = driver3.find_element(By.CLASS_NAME,"prd-brand-info")
        brand_text = brand_info.text
        brandname, product_name, sub_brand_text3 = brand_text.split('\n', 2)
        print(brandname)
    except NoSuchElementException:
            print("No brand info element found.")
            return None
    except TimeoutException:
            print("The product page never loaded.")
            return None
    return brandname

def get_product_name(driver): 
    try:
        brand_info = driver3.find_element(By.CLASS_NAME,"prd-brand-info")
        brand_text = brand_info.text
        brandname, product_name, sub_brand_text3 = brand_text.split('\n', 2)
        print(product_name)
    except NoSuchElementException:
            print("No brand info element found.")
            return None
    except TimeoutException:
            print("The product page never loaded.")
            return None
    return product_name

    # Step 2.2: Extract price
def get_price(driver):
    price = driver3.find_element(By.CLASS_NAME,"sale-price")
    price_text = price.text
    price_start_index = price_text.index("$")+1
    price_end_index = len(price_text)-1
    price_text = float(price_text[price_start_index:price_end_index])
    print(price_text)
    return price_text

    # Step 2.3: Extract rating & review no
def get_rating(driver):
    review_info = driver3.find_element(By.CLASS_NAME,"prd-rating-info")
    review_info_text = review_info.text
    if '\n' in review_info_text:
        rating, r2 = review_info_text.split('\n',1)

        print(f"Rating: "+rating)
        return rating
    else:
        return None
    
    """
def get_rating_review_no(driver):
    review_info = driver3.find_element(By.CLASS_NAME,"prd-rating-info")
    review_info_text = review_info.text
    if '\n' in review_info_text:
        rating, r2 = review_info_text.split('\n',1)
        reviews_no = r2.split(" ")[0]
        if ',' in reviews_no:
            trimmed_reviews_no = int(reviews_no.replace(',',''))
        else: 
            trimmed_reviews_no = int(reviews_no)

        print(f"Rating: "+rating)
        print(f"Reviews Count : {trimmed_reviews_no}")
        return rating, trimmed_reviews_no
    else:
        reviews_no = int(review_info_text.split(" ")[0])
        print(f"Reviews Count : {reviews_no}")
        return None, reviews_no
    """
    # Step 2.??: On 11/21 Olive Young added reviews from Korea, therefore, the review no extracted from "prd-rating-info" is no longer accurate
def get_review_no(driver):
    try:
        review_number = driver3.find_element(By.CLASS_NAME,"review-section-title")
        driver3.execute_script("arguments[0].scrollIntoView(true)",review_number)
        review_number_text = review_number.text
        if "Global Reviews" in review_number_text and "999+" not in review_number_text:
            review_number_text_start_index = review_number_text.index("(")+1
            review_number_text_end_index = len(review_number_text)-1
            review_no = int(review_number_text[review_number_text_start_index:review_number_text_end_index])
            print(f"Review no: {review_no}")
            return review_no
        elif "Global Reviews" in review_number_text & "999+" in review_number_text:
            global_review_no = driver3.find_element(By.CLASS_NAME,"review-list-count").text
            print(f"Review no: {global_review_no}")
            return global_review_no
        else:
            return 0
    except NoSuchElementException:
        print("Cannot find global reviews")
        return None
    

    # Step 2.?: Get category of the product
def get_category(driver):
    try:
        cat_list = driver3.find_elements(By.CLASS_NAME,"loc_cat")
        for i, value in enumerate(cat_list):
            if len(cat_list) > 2:
                if i == 1:
                    cat = value.text
                elif i == 2:
                    subcat = value.text
            elif len(cat_list) <= 2:
                if i == 1:
                    cat = value.text
                    subcat = None
    except NoSuchElementException:
        return None, None
    
    print(f"Category: {cat}")
    print(f"Sub-Category: {subcat}")
    return cat, subcat

    # Step 2.4: Extract ingredients
def get_ingredients(driver):
    item_info_button = driver3.find_element(By.LINK_TEXT,"Specific Item Info")
    item_info_button.click()
    time.sleep(5)
    try:
        item_table = driver3.find_element(By.CLASS_NAME,"prd-desc-tbl")
        ingredients = item_table.find_element(By.XPATH,'//tr[7]/td[1]').text
        print(ingredients)
        driver3.find_element(By.CSS_SELECTOR,".modal-melg > div:nth-child(1) > div:nth-child(1) > button:nth-child(1)").click()
        return ingredients
    except NoSuchElementException:
        driver3.find_element(By.CSS_SELECTOR,".modal-melg > div:nth-child(1) > div:nth-child(1) > button:nth-child(1)").click()
        return None
    
    
    #Step 2.5: Extract reviews 

        # Step 2.5.1: First, find the 'More' button element. Only click if it exists
def get_review_pages(driver):
    def find_more_button(element):
        try:
            return driver3.find_element(By.CLASS_NAME,element)
        except NoSuchElementException:
            return None

    more_page_button = find_more_button(element="review-list-more-btn")
    total_pages = 0
    if more_page_button is not None:
        driver3.execute_script("arguments[0].scrollIntoView(true)",more_page_button)
        pages_text = more_page_button.text
        page_start_index = pages_text.index("/")+1
        page_end_index = len(pages_text)-1
        total_pages = int(pages_text[page_start_index:page_end_index])
        print(f"Total number of review pages: {total_pages}")
        time.sleep(2)
    else:
        total_pages = 0
        print("There are no review pages.")
    
    return total_pages

        #Step 2.5.2: Open up all review pages
def load_review_pages(driver, n):
    for i in range(n-1):
        try:    
            more_page_button = driver3.find_element(By.CLASS_NAME,"review-list-more-btn")
            more_page_button.click()
            time.sleep(5)
            print(f"Clicked {i+1} out of {n} buttons")
            #more_page_button.location_once_scrolled_into_view
            #WebDriverWait(driver3,5).until(EC.element_to_be_clickable((By.CSS_SELECTOR,".btn-page-more")))
                   
        except Exception as e:
            print(f"Exception: {i+1} {e}")
            print("Loading page did not complete.")
            return

        #Step 2.5.3: Translate all reviews (some are in Korean or Japanese)
        # 11/13/24: This function is now obselete. Olive Young removed the translate toggle.
def translate_review(driver):
    try:
        WebDriverWait(driver3,5).until(EC.element_to_be_clickable((By.XPATH,"/html/body/div[5]/div[1]/div[2]/div[3]/div[3]/div/div/div/div[2]/div[1]/div[2]/label"))).click()
    except NoSuchElementException:
        print ("No toggle found.")

        #Step 2.5.4: Expand all reviews 
def expand_review(driver):
    expand_review_buttons = driver3.find_elements(By.CLASS_NAME,"review-list-more-btn")
    print(f"Found {len(expand_review_buttons)} review 'read-more' buttons.")
    for n, button in enumerate(expand_review_buttons):
        try:
            button.click()
            time.sleep(3)
        except Exception as e:
            print(f"Exception : {e} Couldn't click the {n+1}th button.")
            return 


        #Step 2.5.4: Finally extract all review elements       
def extract_review(driver):
    review_list = []
    review = driver3.find_elements(By.CLASS_NAME,"review-unit-cont-comment")
    for n, value in enumerate(review):
        review_text = value.text
        review_list.append(review_text)
    return review_list


             # Extract user skin types  
def extract_user_skin_type(driver):
    skin_type = []
    user_data_attr = driver3.find_elements(By.XPATH,'//dt[text()="Skin Type"]/following-sibling::div/dd')
    for attr in user_data_attr:
        skin_type_attr = attr.text
        if skin_type_attr != '':
            skin_type.append(skin_type_attr)
    print(f"Skin Type found: {skin_type}")
    return skin_type

    
def extract_user_skin_concerns(driver):
    skin_concerns = []
            # Extract user skin concerns
    user_data_attr = driver3.find_elements(By.XPATH,'//dt[text()="Skin Concern"]/following-sibling::div/dd')
    for attr in user_data_attr:
        skin_type_attr = attr.text
        if skin_type_attr != '':
            skin_concerns.append(skin_type_attr)
    print(f"Skin Concerns found: {skin_concerns}")
    return skin_concerns


#Step 0: Set up proxy server
username = "brd-customer-hl_ab73b620-zone-datacenter10"
password = "jz8qvx63tljn"
proxy_host = "brd.superproxy.io"
proxy_port = "22225"

proxy_options = { 
    'proxy': { 
        'http': f'http://{username}:{password}@{proxy_host}:{proxy_port}', 
        'https': f'https://{username}:{password}@{proxy_host}:{proxy_port}', 
        'no_proxy': 'localhost, 127.0.0.1'  # Bypass proxy for local addresses 
    } 
}

driver3 = webdriver.Chrome(seleniumwire_options=proxy_options)

# Step 1: Read the csv file that contains all Olive Young products
#df = pd.read_excel(r'C:\Users\buiva\.vscode\Web Scraping Test/ProductLinks-suncare-split.xlsx', sheet_name='Sheet_5')
#prod_link = df['Link'].tolist()

#  Test a specific Link: 
prod_link = ["https://oliveyoung.com/product/detail?prdtNo=GA240322465"]

products, users = get_product_data(driver3, prod_link)
#users = get_product_data(driver3, prod_link)

#df = pd.DataFrame(products)
#df.to_excel("ProductData-sunscreen-5.xlsx", index = False)

#df = pd.DataFrame(users)
#df.to_excel("UserData-sunscreen-5.xlsx", index = False)

print("Excel files created.")
time.sleep(8)
driver3.quit()

In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_excel(r'C:\Users\buiva\.vscode\Web Scraping Test/ProductLinks-masks.xlsx', sheet_name='ProductLinks-masks')

# Define the chunk size (number of rows per sheet)
chunk_size = 100

# Create a Pandas Excel writer object to write the data into an Excel file
with pd.ExcelWriter("ProductLinks-masks-split.xlsx", engine='xlsxwriter') as writer:
    # Loop through the DataFrame in chunks
    for i in range(0, len(df), chunk_size):
        # Select the chunk of data
        chunk = df.iloc[i:i + chunk_size]
        
        # Write the chunk to a new sheet in the Excel file
        sheet_name = f"Sheet_{i // chunk_size + 1}"  # Naming sheets as Sheet_1, Sheet_2, etc.
        chunk.to_excel(writer, sheet_name=sheet_name, index=False)

print("CSV file has been split into multiple sheets.")